<a href="https://colab.research.google.com/github/saraswatitiwari/Basic-JS/blob/master/rl-baselines-zoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RL Baselines Zoo: Training in Colab



Github Repo: [https://github.com/araffin/rl-baselines-zoo](https://github.com/araffin/rl-baselines-zoo)

Stable-Baselines Repo: [https://github.com/hill-a/stable-baselines](https://github.com/hill-a/stable-baselines)

Medium article: [https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82](https://medium.com/@araffin/stable-baselines-a-fork-of-openai-baselines-df87c4b2fc82)

# Install Dependencies



In [5]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!apt-get update
!apt-get install swig cmake libopenmpi-dev zlib1g-dev ffmpeg freeglut3-dev xvfb
!pip install stable-baselines[mpi] --upgrade
!pip install pybullet
!pip install box2d box2d-kengz pyyaml pytablewriter optuna scikit-optimize


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

## Clone RL Baselines Zoo Repo

In [6]:
!git clone https://github.com/araffin/rl-baselines-zoo

Cloning into 'rl-baselines-zoo'...
remote: Enumerating objects: 1843, done.
remote: Counting objects: 100% (488/488), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 1843 (delta 462), reused 443 (delta 443), pack-reused 1355
Receiving objects: 100% (1843/1843), 375.66 MiB | 34.62 MiB/s, done.
Resolving deltas: 100% (1111/1111), done.
Checking out files: 100% (333/333), done.


In [ ]:
cd rl-baselines-zoo/

/content/rl-baselines-zoo


## Train an RL Agent


The train agent can be found in the `logs/` folder.

Here we will train A2C on CartPole-v1 environment for 100 000 steps. 


To train it on Pong (Atari), you just have to pass `--env PongNoFrameskip-v4`

Note: You need to update `hyperparams/algo.yml` to support new environments. You can access it in the side panel of Google Colab. (see https://stackoverflow.com/questions/46986398/import-data-into-google-colaboratory)

In [7]:
!python train.py --algo a2c --env CartPole-v1 --n-timesteps 100000

python3: can't open file 'train.py': [Errno 2] No such file or directory


#### Evaluate trained agent


You can remove the `--folder logs/` to evaluate pretrained agent.

In [8]:
!python enjoy.py --algo a2c --env CartPole-v1 --no-render --n-timesteps 5000 --folder logs/

python3: can't open file 'enjoy.py': [Errno 2] No such file or directory


#### Tune Hyperparameters

We use [Optuna](https://optuna.org/) for optimizing the hyperparameters.

Tune the hyperparameters for PPO2, using a tpe sampler and median pruner, 2 parallels jobs,
with a budget of 1000 trials and a maximum of 50000 steps

In [9]:
!python -m train.py --algo ppo2 --env MountainCar-v0 -n 50000 -optimize --n-trials 1000 --n-jobs 2 --sampler tpe --pruner median

/usr/bin/python3: Error while finding module specification for 'train.py' (ModuleNotFoundError: No module named 'train')


### Record  a Video

In [10]:
# Set up display; otherwise rendering will fail
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [11]:
!pip install pyglet==1.3.1  # pyglet v1.4.1 throws an error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.6 MB/s 
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires pyglet<=1.5.0,>=1.4.0, but you have pyglet 1.3.1 which is incompatible.


In [12]:
!python -m utils.record_video --algo a2c --env CartPole-v1 --exp-id 0 -f logs/ -n 1000

/usr/bin/python3: Error while finding module specification for 'utils.record_video' (ModuleNotFoundError: No module named 'utils')


### Display the video

In [13]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay

def show_videos(video_path='', prefix=''):
  """
  Taken from https://github.com/eleurent/highway-env

  :param video_path: (str) Path to the folder containing videos
  :param prefix: (str) Filter the video, showing only the only starting with this prefix
  """
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

In [14]:
show_videos(prefix='a2c')

### Continue Training

Here, we will continue training of the previous model

In [15]:
!python train.py --algo a2c --env CartPole-v1 --n-timesteps 50000 -i logs/a2c/CartPole-v1.zip

python3: can't open file 'train.py': [Errno 2] No such file or directory


In [16]:
!python enjoy.py --algo a2c --env CartPole-v1 --no-render --n-timesteps 1000 --folder logs/

python3: can't open file 'enjoy.py': [Errno 2] No such file or directory
